# New York Taxi Data Wrangling
### Data Engineering Capstone Project

![](taxi.png)

#### Project Summary
In this project, an ETL pipeline is developed to ready data of New York Taxi trips
and hourly weather for analysts to analyse the prepared data.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
After the final data is stored in parquet files in an S3 bucket for the analysts,
the analysts can analyse how the passengers pay, trips depending on weather conditions,
fare breakup and final amount based on the trip route, weather, service type etc.

#### Data Sources
* Taxi Trips -
	* [Source](https://data.cityofnewyork.us/Transportation/2020-Green-Taxi-Trip-Data-January-June-/pkmi-4kfn)
	* Format - CSV
	* Type of Info - These records are generated from the trip record submissions made
	by green taxi Technology Service Providers (TSPs). Each row represents a single trip.
	The trip records include fields capturing pick-up and drop-off dates/times, pick-up
	and drop-off locations, trip distances, itemized fares, rate types, payment types,
	and driver-reported passenger counts, etc.

* Zone Info -
	* [Source](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
	* Format - JSON
	* Type of Info - These records are the pick-up or drop-off locations across New York
	where the taxis ply.


* Hourly Weather -
	* [Source](https://www.ncdc.noaa.gov/cdo-web/orders?email=arnav.gohil04@gmail.com&id=2393740)
	* Format - CSV
	* Type of Info - Each record represents hourly weather data recorded across various
	stations in New York. Each record contains different weather parameters like Clouds
	broken percentage, Temperature 90th percentile, Wind Chill, Wind direction, etc.

In [2]:
# Reading the data from various sources and putting it into Panda dataframe to validate it

taxi_file_url = "https://data.cityofnewyork.us/api/views/pkmi-4kfn/rows.csv"
taxi_zone_file = "taxi_zones.json"
weather_file = "weather.csv"

df_zones = pd.read_json(taxi_zone_file)
df_weather = pd.read_csv(weather_file)
df_taxi = pd.read_csv(taxi_file_url)

/Users/arnavgohil/.conda/envs/Capstone-Project/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_taxi.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,12/18/2019 03:52:30 PM,12/18/2019 03:54:39 PM,N,1.0,264,264,5.0,0.00,3.5,0.50,0.5,0.01,0.0,NaN,0.3,4.81,1.0,1.0,0.00
1,2.0,01/01/2020 12:45:58 AM,01/01/2020 12:56:39 AM,N,5.0,66,65,2.0,1.28,20.0,0.00,0.0,4.06,0.0,NaN,0.3,24.36,1.0,2.0,0.00
2,2.0,01/01/2020 12:41:38 AM,01/01/2020 12:52:49 AM,N,1.0,181,228,1.0,2.47,10.5,0.50,0.5,3.54,0.0,NaN,0.3,15.34,1.0,1.0,0.00
3,1.0,01/01/2020 12:52:46 AM,01/01/2020 01:14:21 AM,N,1.0,129,263,2.0,6.30,21.0,3.25,0.5,0.00,0.0,NaN,0.3,25.05,2.0,1.0,2.75
4,1.0,01/01/2020 12:19:57 AM,01/01/2020 12:30:56 AM,N,1.0,210,150,1.0,2.30,10.0,0.50,0.5,0.00,0.0,NaN,0.3,11.30,1.0,1.0,0.00


In [4]:
df_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [5]:
df_weather.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,HLY-CLDH-NORMAL,HLY-CLOD-PCTBKN,HLY-CLOD-PCTCLR,HLY-CLOD-PCTFEW,...,HLY-TEMP-NORMAL,HLY-WCHL-NORMAL,HLY-WIND-1STDIR,HLY-WIND-1STPCT,HLY-WIND-2NDDIR,HLY-WIND-2NDPCT,HLY-WIND-AVGSPD,HLY-WIND-PCTCLM,HLY-WIND-VCTDIR,HLY-WIND-VCTSPD
0,USW00014732,"LA GUARDIA AIRPORT, NY US",40.7792,-73.88,3.4,01-01T01:00:00,0.0,134,268,75,...,1.4,-3.2,8,269,6,178,5.6,24,308,2.8
1,USW00014732,"LA GUARDIA AIRPORT, NY US",40.7792,-73.88,3.4,01-01T02:00:00,0.0,155,277,79,...,1.2,-3.4,8,251,7,197,5.6,33,309,2.9
2,USW00014732,"LA GUARDIA AIRPORT, NY US",40.7792,-73.88,3.4,01-01T03:00:00,0.0,162,277,63,...,1.1,-3.6,8,282,7,182,5.5,27,312,2.9
3,USW00014732,"LA GUARDIA AIRPORT, NY US",40.7792,-73.88,3.4,01-01T04:00:00,0.0,149,262,68,...,0.9,-3.8,8,284,7,164,5.5,29,314,2.9
4,USW00014732,"LA GUARDIA AIRPORT, NY US",40.7792,-73.88,3.4,01-01T05:00:00,0.0,135,243,119,...,0.8,-3.9,8,273,7,174,5.5,31,314,2.9


In [6]:
#Creating Rate Panda dataframe

rate_data = { 'RatecodeID':[1,2,3,4,5,6],
              'Rate_type': ['Standard rate','JFK','Newark','Nassau or Westchester','Negotiated fare','Group ride']}
df_rate = pd.DataFrame(rate_data)
df_rate.head()

,RatecodeID,Rate_type
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare


In [7]:
#Creating payment Panda dataframe

payment_data = { 'PaymentID':[1,2,3,4,5,6],
                 'Payment_type': ['Credit card','Cash','No charge','Dispute','Unknown','Voided trip']}
df_payment = pd.DataFrame(payment_data)
df_payment.head()

,PaymentID,Payment_type
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown


In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import datetime

In [11]:
#Creating a new Spark Session

MAX_MEMORY = "5g"
spark = SparkSession \
        .builder \
        .config("spark.executor.memory", MAX_MEMORY) \
        .config("spark.driver.memory", MAX_MEMORY) \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [12]:
spark

In [13]:
#Creating a Weather Spark Dataframe

weather_spark_df = spark.read.option("multiline","true").option("header", "true").csv(weather_file)
weather_spark_df.show(10)

+-----------+--------------------+--------+---------+---------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|    STATION|                NAME|LATITUDE|LONGITUDE|ELEVATION|          DATE|HLY-CLDH-NORMAL|HLY-CLOD-PCTBKN|HLY-CLOD-PCTCLR|HLY-CLOD-PCTFEW|HLY-CLOD-PCTOVC|HLY-CLOD-PCTSCT|HLY-DEWP-10PCTL|HLY-DEWP-90PCTL|HLY-DEWP-NORMAL|HLY-HIDX-NORMAL|HLY-HTDH-NORMAL|HLY-PRES-10PCTL|HLY-PRES-90PCTL|HLY-PRES-NORMAL|HLY-TEMP-10PCTL|HLY-TEMP-90PCTL|HLY-TEMP-NORMAL|HLY-WCHL-NORMAL|HLY-WIND-1STDIR|HLY-WIND-1STPCT|HLY-WIND-2NDDIR|HLY-WIND-2NDPCT|HLY-WIND-AVGSPD|HLY-WIND-PCTCLM|HLY-WIND-VCTDIR|HLY-WIND-VCTSPD|
+---------

In [14]:
#Creating a Zone Spark Dataframe

zone_spark_df = spark.read.option("multiline","true").json(taxi_zone_file)
zone_spark_df.show(1)

+-------+----------+--------------+------------+
|Borough|LocationID|          Zone|service_zone|
+-------+----------+--------------+------------+
|    EWR|         1|Newark Airport|         EWR|
+-------+----------+--------------+------------+
only showing top 1 row



In [15]:
df_taxi.head(2)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,12/18/2019 03:52:30 PM,12/18/2019 03:54:39 PM,N,1.0,264,264,5.0,0.00,3.5,0.5,0.5,0.01,0.0,NaN,0.3,4.81,1.0,1.0,0.0
1,2.0,01/01/2020 12:45:58 AM,01/01/2020 12:56:39 AM,N,5.0,66,65,2.0,1.28,20.0,0.0,0.0,4.06,0.0,NaN,0.3,24.36,1.0,2.0,0.0


In [16]:
#Creating a new struct for taxi data so that data from the existing Panda DF can be
#copied instead of downloading it again

from pyspark.sql.types import *

taxiSchema = StructType([StructField("VendorID", FloatType(), True)
                       ,StructField("lpep_pickup_datetime", StringType(), True)
                       ,StructField("lpep_dropoff_datetime", StringType(), True)
                       ,StructField("store_and_fwd_flag", StringType(), True)
                       ,StructField("RatecodeID", FloatType(), True)
                       ,StructField("PULocationID", IntegerType(), True)
                       ,StructField("DOLocationID", IntegerType(), True)
                       ,StructField("passenger_count", FloatType(), True)
                       ,StructField("trip_distance", FloatType(), True)
                       ,StructField("fare_amount", FloatType(), True)
                       ,StructField("extra", FloatType(), True)
                       ,StructField("mta_tax", FloatType(), True)
                       ,StructField("tip_amount", FloatType(), True)
                       ,StructField("tolls_amount", FloatType(), True)
                       ,StructField("ehail_fee", FloatType(), True)
                       ,StructField("improvement_surcharge", FloatType(), True)
                       ,StructField("total_amount", FloatType(), True)
                       ,StructField("payment_type", FloatType(), True)
                       ,StructField("trip_type", FloatType(), True)
                       ,StructField("congestion_surcharge", FloatType(), True)])


In [17]:
#Creating a taxi Spark Dataframe from the existing Panda DF
taxi_spark_df = spark.createDataFrame(df_taxi,schema=taxiSchema)
taxi_spark_df.show(2)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|     2.0|12/18/2019 03:52:...| 12/18/2019 03:54:...|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [18]:
#Creating a Rate Spark Dataframe from the existing Panda DF

rateSchema = StructType([StructField("RatecodeID",IntegerType(),True),StructField("Rate_type",StringType(),True)])
rate_spark_df = spark.createDataFrame(df_rate,schema=rateSchema)
rate_spark_df.head(2)

[Row(RatecodeID=1, Rate_type='Standard rate'),
 Row(RatecodeID=2, Rate_type='JFK')]

In [19]:
#Creating a Rate Spark Dataframe from the existing Panda DF

paymentSchema = StructType([StructField("PaymentID",IntegerType(),True),StructField("Payment_type",StringType(),True)])
payment_spark_df = spark.createDataFrame(df_payment,schema=paymentSchema)
payment_spark_df.show(2)

+---------+------------+
|PaymentID|Payment_type|
+---------+------------+
|        1| Credit card|
|        2|        Cash|
+---------+------------+
only showing top 2 rows



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

1. Remove rows where Payment ID is Unknown .
2. Remove rows where PickUpID or DropID is 264 or 265, check all values in range .
3. Convert PickUpTime and DropTime to TIMESTAMP .
4. Remove rows which are out of range in taxi  .
5. Remove rows with Missing Columns in taxi data or is NULL/NaN .
6. Remove duplicate data in weather. (keep only JFK, remove others) .
7. Convert DATE and DropTime to TIMESTAMP .
8. Filter Weather data so that it matches time period of taxi data .

#### Cleaning Steps
Document steps necessary to clean the data

In [20]:
#taxi_spark_df.count()

In [21]:
# Performing cleaning task 1
taxi_spark_df = taxi_spark_df.filter(taxi_spark_df["payment_type"] != 5)

In [22]:
# Performing cleaning task 2
taxi_spark_df = taxi_spark_df.filter(taxi_spark_df["PULocationID"] != 264)\
                             .filter(taxi_spark_df["PULocationID"] != 265)

In [23]:
# Performing cleaning task 3

# add column with new time stamp column (x2)
taxi_spark_df = taxi_spark_df.withColumn("PickupTS",f.to_timestamp(taxi_spark_df["lpep_pickup_datetime"],"MM/dd/yyyy hh:mm:ss a"))
taxi_spark_df = taxi_spark_df.withColumn("DropTS",f.to_timestamp(taxi_spark_df["lpep_dropoff_datetime"],"MM/dd/yyyy hh:mm:ss a"))
# drop column with new time stamp column (x2)
taxi_spark_df = taxi_spark_df.drop("lpep_pickup_datetime","lpep_dropoff_datetime")

taxi_spark_df.head(10)

[Row(VendorID=2.0, store_and_fwd_flag='N', RatecodeID=5.0, PULocationID=66, DOLocationID=65, passenger_count=2.0, trip_distance=1.2799999713897705, fare_amount=20.0, extra=0.0, mta_tax=0.0, tip_amount=4.059999942779541, tolls_amount=0.0, ehail_fee=nan, improvement_surcharge=0.30000001192092896, total_amount=24.360000610351562, payment_type=1.0, trip_type=2.0, congestion_surcharge=0.0, PickupTS=datetime.datetime(2020, 1, 1, 0, 45, 58), DropTS=datetime.datetime(2020, 1, 1, 0, 56, 39)),
 Row(VendorID=2.0, store_and_fwd_flag='N', RatecodeID=1.0, PULocationID=181, DOLocationID=228, passenger_count=1.0, trip_distance=2.4700000286102295, fare_amount=10.5, extra=0.5, mta_tax=0.5, tip_amount=3.5399999618530273, tolls_amount=0.0, ehail_fee=nan, improvement_surcharge=0.30000001192092896, total_amount=15.34000015258789, payment_type=1.0, trip_type=1.0, congestion_surcharge=0.0, PickupTS=datetime.datetime(2020, 1, 1, 0, 41, 38), DropTS=datetime.datetime(2020, 1, 1, 0, 52, 49)),
 Row(VendorID=1.0, s

In [24]:
# Performing cleaning task 4
# filter data on the basis of timestamp
from_date = datetime.datetime(2020,1,1,0,0,0)
to_date = datetime.datetime(2020,6,30,23,59,59)

taxi_spark_df = taxi_spark_df.filter(taxi_spark_df["PickupTS"] >= from_date)\
                             .filter(taxi_spark_df["DropTS"] <= to_date)

In [25]:
# Performing cleaning task 5
taxi_spark_df = taxi_spark_df.filter(taxi_spark_df["PickupTS"].isNotNull() == True)\
                             .filter(taxi_spark_df["DropTS"].isNotNull() == True)\
                             .filter(taxi_spark_df["RatecodeID"].isNotNull() == True)\
                             .filter(taxi_spark_df["PULocationID"].isNotNull() == True)\
                             .filter(taxi_spark_df["DOLocationID"].isNotNull() == True)\
                             .filter(taxi_spark_df["passenger_count"].isNotNull() == True)\
                             .filter(taxi_spark_df["trip_distance"].isNotNull() == True)\
                             .filter(taxi_spark_df["fare_amount"].isNotNull() == True)\
                             .filter(taxi_spark_df["payment_type"].isNotNull() == True)\
                             .filter(taxi_spark_df["extra"].isNotNull() == True)\
                             .filter(taxi_spark_df["mta_tax"].isNotNull() == True)\
                             .filter(taxi_spark_df["tip_amount"].isNotNull() == True)\
                             .filter(taxi_spark_df["tolls_amount"].isNotNull() == True)\
                             .filter(taxi_spark_df["improvement_surcharge"].isNotNull() == True)\
                             .filter(taxi_spark_df["total_amount"].isNotNull() == True)\
                             .filter(taxi_spark_df["congestion_surcharge"].isNotNull() == True)

In [26]:
# Performing cleaning task 6
weather_spark_df = weather_spark_df.filter(weather_spark_df['name'] == "JFK INTERNATIONAL AIRPORT, NY US")
weather_spark_df.head(30)

[Row(STATION='USW00094789', NAME='JFK INTERNATIONAL AIRPORT, NY US', LATITUDE='40.6386', LONGITUDE='-73.7622', ELEVATION='3.4', DATE='01-01T01:00:00', HLY-CLDH-NORMAL='0.0', HLY-CLOD-PCTBKN='116', HLY-CLOD-PCTCLR='240', HLY-CLOD-PCTFEW='119', HLY-CLOD-PCTOVC='445', HLY-CLOD-PCTSCT='80', HLY-DEWP-10PCTL='-15.6', HLY-DEWP-90PCTL='5.6', HLY-DEWP-NORMAL='-4.6', HLY-HIDX-NORMAL='0.9', HLY-HTDH-NORMAL='17.5', HLY-PRES-10PCTL='1005.2', HLY-PRES-90PCTL='1030.8', HLY-PRES-NORMAL='1018.2', HLY-TEMP-10PCTL='-6.1', HLY-TEMP-90PCTL='7.2', HLY-TEMP-NORMAL='0.9', HLY-WCHL-NORMAL='-3.7', HLY-WIND-1STDIR='8', HLY-WIND-1STPCT='258', HLY-WIND-2NDDIR='7', HLY-WIND-2NDPCT='204', HLY-WIND-AVGSPD='5.3', HLY-WIND-PCTCLM='33', HLY-WIND-VCTDIR='304', HLY-WIND-VCTSPD='2.8'),
 Row(STATION='USW00094789', NAME='JFK INTERNATIONAL AIRPORT, NY US', LATITUDE='40.6386', LONGITUDE='-73.7622', ELEVATION='3.4', DATE='01-01T02:00:00', HLY-CLDH-NORMAL='0.0', HLY-CLOD-PCTBKN='131', HLY-CLOD-PCTCLR='235', HLY-CLOD-PCTFEW='124'

In [27]:
#TODO Fix
# Performing cleaning task 7
# write udf
from pyspark.sql.functions import udf,to_timestamp
format = udf(lambda x : x.replace("T"," ") + " 2020")
# add column
weather_spark_df = weather_spark_df.withColumn("TIME",format(weather_spark_df["DATE"]))
weather_spark_df = weather_spark_df.withColumn("ts", to_timestamp(weather_spark_df["TIME"],"MM-dd HH:mm:ss yyyy"))
# delete old column
weather_spark_df = weather_spark_df.drop("DATE")
weather_spark_df = weather_spark_df.drop("TIME")

weather_spark_df.head(30)

[Row(STATION='USW00094789', NAME='JFK INTERNATIONAL AIRPORT, NY US', LATITUDE='40.6386', LONGITUDE='-73.7622', ELEVATION='3.4', HLY-CLDH-NORMAL='0.0', HLY-CLOD-PCTBKN='116', HLY-CLOD-PCTCLR='240', HLY-CLOD-PCTFEW='119', HLY-CLOD-PCTOVC='445', HLY-CLOD-PCTSCT='80', HLY-DEWP-10PCTL='-15.6', HLY-DEWP-90PCTL='5.6', HLY-DEWP-NORMAL='-4.6', HLY-HIDX-NORMAL='0.9', HLY-HTDH-NORMAL='17.5', HLY-PRES-10PCTL='1005.2', HLY-PRES-90PCTL='1030.8', HLY-PRES-NORMAL='1018.2', HLY-TEMP-10PCTL='-6.1', HLY-TEMP-90PCTL='7.2', HLY-TEMP-NORMAL='0.9', HLY-WCHL-NORMAL='-3.7', HLY-WIND-1STDIR='8', HLY-WIND-1STPCT='258', HLY-WIND-2NDDIR='7', HLY-WIND-2NDPCT='204', HLY-WIND-AVGSPD='5.3', HLY-WIND-PCTCLM='33', HLY-WIND-VCTDIR='304', HLY-WIND-VCTSPD='2.8', ts=datetime.datetime(2020, 1, 1, 1, 0)),
 Row(STATION='USW00094789', NAME='JFK INTERNATIONAL AIRPORT, NY US', LATITUDE='40.6386', LONGITUDE='-73.7622', ELEVATION='3.4', HLY-CLDH-NORMAL='0.0', HLY-CLOD-PCTBKN='131', HLY-CLOD-PCTCLR='235', HLY-CLOD-PCTFEW='124', HLY-

In [28]:
# Performing cleaning task 8
# filter data on the basis of timestamp
weather_spark_df = weather_spark_df.filter(weather_spark_df["ts"] >= from_date)\
                             .filter(weather_spark_df["ts"] <= to_date)


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [29]:
import configparser,os
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

output_data = "s3a://my-udacity-course-bucket/capstone-project/"

In [30]:
#Rate_Table
#rate_spark_df.write.parquet(output_data + "/rate_table/",mode="overwrite")

In [31]:
#Payment_Table
#payment_spark_df.write.parquet(output_data + "/payment_table/",mode="overwrite")

In [32]:
#Location_Table
zone_table = zone_spark_df.select(["LocationID","Borough","Zone"])
#zone_table.write.parquet(output_data + "/zone_table/",mode="overwrite")
zone_table.show(10)

+----------+-------------+--------------------+
|LocationID|      Borough|                Zone|
+----------+-------------+--------------------+
|         1|          EWR|      Newark Airport|
|         2|       Queens|         Jamaica Bay|
|         3|        Bronx|Allerton/Pelham G...|
|         4|    Manhattan|       Alphabet City|
|         5|Staten Island|       Arden Heights|
|         6|Staten Island|Arrochar/Fort Wad...|
|         7|       Queens|             Astoria|
|         8|       Queens|        Astoria Park|
|         9|       Queens|          Auburndale|
|        10|       Queens|        Baisley Park|
+----------+-------------+--------------------+
only showing top 10 rows



In [33]:
weather_spark_df = weather_spark_df.withColumnRenamed("HLY-CLOD-PCTBKN","Clouds_Broken_Percentage") \
                                    .withColumnRenamed("HLY-DEWP-NORMAL","Dew_Point") \
                                    .withColumnRenamed("HLY-HTDH-NORMAL","Heating_Degree_Hours") \
                                    .withColumnRenamed("HLY-HIDX-NORMAL","Heat_Index_Mean") \
                                    .withColumnRenamed("HLY-WIND-AVGSPD","Wind_Speed") \
                                    .withColumnRenamed("HLY-WIND-PCTCLM","Percentage_Calm") \
                                    .withColumnRenamed("HLY-CLOD-PCTOVC","Clouds_Overcast_Percentage") \
                                    .withColumnRenamed("HLY-PRES-NORMAL","Sea_Level_Pressure") \
                                    .withColumnRenamed("HLY-TEMP-NORMAL","Temperature") \
                                    .withColumnRenamed("HLY-WCHL-NORMAL","Wind_Chill") \
                                    .withColumnRenamed("HLY-WIND-1STDIR","Wind_Direction")

weather_spark_df.show(10)

+-----------+--------------------+--------+---------+---------+---------------+------------------------+---------------+---------------+--------------------------+---------------+---------------+---------------+---------+---------------+--------------------+---------------+---------------+------------------+---------------+---------------+-----------+----------+--------------+---------------+---------------+---------------+----------+---------------+---------------+---------------+-------------------+
|    STATION|                NAME|LATITUDE|LONGITUDE|ELEVATION|HLY-CLDH-NORMAL|Clouds_Broken_Percentage|HLY-CLOD-PCTCLR|HLY-CLOD-PCTFEW|Clouds_Overcast_Percentage|HLY-CLOD-PCTSCT|HLY-DEWP-10PCTL|HLY-DEWP-90PCTL|Dew_Point|Heat_Index_Mean|Heating_Degree_Hours|HLY-PRES-10PCTL|HLY-PRES-90PCTL|Sea_Level_Pressure|HLY-TEMP-10PCTL|HLY-TEMP-90PCTL|Temperature|Wind_Chill|Wind_Direction|HLY-WIND-1STPCT|HLY-WIND-2NDDIR|HLY-WIND-2NDPCT|Wind_Speed|Percentage_Calm|HLY-WIND-VCTDIR|HLY-WIND-VCTSPD|       

In [34]:
weather_spark_df.createOrReplaceTempView("weather_view")

spark.udf.register("getMonth",lambda x : int(x.month))
spark.udf.register("getDay",lambda x : int(x.day))
spark.udf.register("getHour",lambda x : int(x.hour))
spark.udf.register("getTS",lambda x : int(x.timestamp))

<function __main__.<lambda>(x)>

In [35]:
#Weather_Table
weather_table = spark.sql("""
    SELECT
        Clouds_Broken_Percentage,
        Dew_Point,
        Heating_Degree_Hours,
        Heat_Index_Mean,
        Wind_Speed,
        Percentage_Calm,
        Clouds_Overcast_Percentage,
        Sea_Level_Pressure,
        Temperature,
        Wind_Chill,
        Wind_Direction,
        getMonth(ts) AS Month,
        getDay(ts) AS Day,
        getHour(ts) AS Hour
    FROM weather_view;
""")

#weather_table.write.partitionBy("month","day").parquet(output_data + "/zone_table/",mode="overwrite")
weather_table.show(10)

+------------------------+---------+--------------------+---------------+----------+---------------+--------------------------+------------------+-----------+----------+--------------+-----+---+----+
|Clouds_Broken_Percentage|Dew_Point|Heating_Degree_Hours|Heat_Index_Mean|Wind_Speed|Percentage_Calm|Clouds_Overcast_Percentage|Sea_Level_Pressure|Temperature|Wind_Chill|Wind_Direction|Month|Day|Hour|
+------------------------+---------+--------------------+---------------+----------+---------------+--------------------------+------------------+-----------+----------+--------------+-----+---+----+
|                     116|     -4.6|                17.5|            0.9|       5.3|             33|                       445|            1018.2|        0.9|      -3.7|             8|    1|  1|   1|
|                     131|     -4.6|                17.6|            0.8|       5.2|             40|                       443|            1018.4|        0.8|      -3.7|             8|    1|  1|   2|


In [36]:
taxi_spark_df = taxi_spark_df.withColumn("id",f.monotonically_increasing_id())
taxi_spark_df.createOrReplaceTempView("taxi_view")
taxi_spark_df.show(10)

+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------------------+-------------------+---+
|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|           PickupTS|             DropTS| id|
+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------------------+-------------------+---+
|     2.0|                 N|       5.0|          66|          65|            2.0|         1.28|       20.0|  0.0|    0.0|      4.06|         0.0|      NaN|    

In [37]:
spark.udf.register("getTS",lambda x : datetime.datetime.timestamp(x))

#Trip_table
trip_table = spark.sql("""
    SELECT
        id AS trip_id,
        getTS(PickupTS) AS pickup_timestamp,
        getTS(DropTS) AS drop_timestamp,
        fare_amount,
        extra,
        tip_amount,
        tolls_amount,
        congestion_surcharge,
        total_amount
    FROM taxi_view""")

#trip_table.write.parquet(output_data + "/trip_table/",mode="overwrite")
trip_table.show(10)

+-------+----------------+--------------+-----------+-----+----------+------------+--------------------+------------+
|trip_id|pickup_timestamp|drop_timestamp|fare_amount|extra|tip_amount|tolls_amount|congestion_surcharge|total_amount|
+-------+----------------+--------------+-----------+-----+----------+------------+--------------------+------------+
|      0|   1.577819758E9| 1.577820399E9|       20.0|  0.0|      4.06|         0.0|                 0.0|       24.36|
|      1|   1.577819498E9| 1.577820169E9|       10.5|  0.5|      3.54|         0.0|                 0.0|       15.34|
|      2|   1.577820166E9| 1.577821461E9|       21.0| 3.25|       0.0|         0.0|                2.75|       25.05|
|      3|   1.577818197E9| 1.577818856E9|       10.0|  0.5|       0.0|         0.0|                 0.0|        11.3|
|      4|   1.577820153E9| 1.577821194E9|       13.5|  0.5|       0.0|         0.0|                 0.0|        14.8|
|      5|   1.577817618E9| 1.577818336E9|       11.0|  0

In [38]:
#Fact_table
fact_table = spark.sql("""
    SELECT
        id AS trip_id,
        getMonth(PickupTS) AS Month,
        getDay(PickupTS) AS Day,
        getHour(PickupTS) AS Hour,
        RatecodeID AS rate_code,
        payment_type AS payment_code,
        PULocationID AS pick_up_loc_id,
        DOLocationID AS drop_loc_id,
        total_amount,
        passenger_count,
        trip_distance
    FROM  taxi_view""")

#fact_table.write.partitionBy("Month","Day","Hour").parquet(output_data + "/fact_table/",mode="overwrite")
fact_table.show(10)

+-------+-----+---+----+---------+------------+--------------+-----------+------------+---------------+-------------+
|trip_id|Month|Day|Hour|rate_code|payment_code|pick_up_loc_id|drop_loc_id|total_amount|passenger_count|trip_distance|
+-------+-----+---+----+---------+------------+--------------+-----------+------------+---------------+-------------+
|      0|    1|  1|   0|      5.0|         1.0|            66|         65|       24.36|            2.0|         1.28|
|      1|    1|  1|   0|      1.0|         1.0|           181|        228|       15.34|            1.0|         2.47|
|      2|    1|  1|   0|      1.0|         2.0|           129|        263|       25.05|            2.0|          6.3|
|      3|    1|  1|   0|      1.0|         1.0|           210|        150|        11.3|            1.0|          2.3|
|      4|    1|  1|   0|      1.0|         1.0|            35|         39|        14.8|            1.0|          3.0|
|      5|    1|  1|   0|      1.0|         2.0|         

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness

1. Check for null value of ID in fact
2. Check for null value of ID in trip
3. Check if join works fine (fact <-> trip)
4. Check for null value of month,day,hour in fact
5. Check for null value of month,day,hour in Weather
6. Check if join works fine (fact <-> weather)
7. Check if join works fine (fact <-> rate)
8. Check if join works fine (fact <-> location)
9. Check if join works fine (fact <-> payment)

Run Quality Checks

In [39]:
trip_table.createOrReplaceTempView("trip_view")
fact_table.createOrReplaceTempView("fact_view")
weather_table.createOrReplaceTempView("weather_view")
zone_table.createOrReplaceTempView("zone_view")
payment_spark_df.createOrReplaceTempView("payment_view")
rate_spark_df.createOrReplaceTempView("rate_view")


In [57]:
#Task 1
spark.sql("SELECT COUNT(*) FROM fact_view").show(1)

+--------+
|count(1)|
+--------+
| 1222824|
+--------+



In [40]:
#Task 2
spark.sql("SELECT COUNT(*) FROM fact_view WHERE trip_id is NULL ").show(1)

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [41]:
#Task 3
spark.sql("SELECT COUNT(*) FROM trip_view WHERE trip_id is NULL ").show(1)

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [55]:
#Task 4
spark.sql("""
    SELECT *
    FROM fact_view F
    JOIN trip_view T
    ON F.trip_id = T.trip_id""").show(10)

In [53]:
#Task 5
spark.sql("""SELECT COUNT(*)
             FROM fact_view
             WHERE month is NULL
             OR hour is NULL
             OR day is NULL""").show(1)

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [51]:
#Task 6
spark.sql("""SELECT COUNT(*)
             FROM weather_view
             WHERE month is NULL
             OR hour is NULL
             OR day is NULL""").show(1)


+--------+
|count(1)|
+--------+
|       0|
+--------+



In [52]:
#Task 7
spark.sql("""
    SELECT
        F.trip_id, W.*
    FROM fact_view F
    JOIN weather_view W
    ON F.month = W.month
    AND F.hour = W.hour
    AND F.day = W.day""").show(10)


+-------+------------------------+---------+--------------------+---------------+----------+---------------+--------------------------+------------------+-----------+----------+--------------+-----+---+----+
|trip_id|Clouds_Broken_Percentage|Dew_Point|Heating_Degree_Hours|Heat_Index_Mean|Wind_Speed|Percentage_Calm|Clouds_Overcast_Percentage|Sea_Level_Pressure|Temperature|Wind_Chill|Wind_Direction|Month|Day|Hour|
+-------+------------------------+---------+--------------------+---------------+----------+---------------+--------------------------+------------------+-----------+----------+--------------+-----+---+----+
|      6|                     116|     -4.6|                17.5|            0.9|       5.3|             33|                       445|            1018.2|        0.9|      -3.7|             8|    1|  1|   1|
|    133|                     116|     -4.6|                17.5|            0.9|       5.3|             33|                       445|            1018.2|        0.9|  

In [50]:
#Task 8
spark.sql("""
    SELECT
        COUNT(*), R.Rate_type
    FROM fact_view F
    JOIN rate_view R
    ON F.rate_code = R.RatecodeID
    GROUP BY R.Rate_type""").show(10)


+--------+--------------------+
|count(1)|           Rate_type|
+--------+--------------------+
|   22836|     Negotiated fare|
|     645|Nassau or Westche...|
|     435|              Newark|
|  891842|       Standard rate|
|      10|          Group ride|
|    1731|                 JFK|
+--------+--------------------+



In [49]:
#Task 9
spark.sql("""
    SELECT
        Z1.id, Z1.Pick_Zone, Z2.Drop_Zone
    FROM
        (SELECT F.trip_id AS ID, Z.Zone AS Pick_Zone FROM fact_view F JOIN zone_view Z ON F.pick_up_loc_id = Z.LocationID ) Z1
    JOIN
        (SELECT F.trip_id AS ID, Z.Zone AS Drop_Zone FROM fact_view F JOIN zone_view Z ON F.drop_loc_id = Z.LocationID ) Z2
    ON  Z1.ID = Z2.ID
    """).show(10)

+----+--------------------+--------------------+
|  id|           Pick_Zone|           Drop_Zone|
+----+--------------------+--------------------+
|  26|         Kew Gardens|     Jackson Heights|
|  29|Washington Height...|Washington Height...|
| 474|      Central Harlem|    Manhattan Valley|
| 964|   East Harlem South|   East Harlem South|
|1677|             Astoria|            Elmhurst|
|1697|             Astoria|       East Elmhurst|
|1806| Crown Heights North|    Sunset Park West|
|1950|        Clinton Hill|            Gramercy|
|2040|    Bensonhurst West|       Starrett City|
|2214|            Elmhurst|        North Corona|
+----+--------------------+--------------------+
only showing top 10 rows



In [56]:
#Task 10
spark.sql("""
    SELECT
        COUNT(*), P.Payment_type
    FROM fact_view F
    JOIN payment_view P
    ON F.payment_code = P.PaymentID
    GROUP BY P.Payment_type""").show(10)



+--------+------------+
|count(1)|Payment_type|
+--------+------------+
|  409664|        Cash|
|    1682|     Dispute|
|    5397|   No charge|
|  500758| Credit card|
+--------+------------+



#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.